In [1]:
import pandas as pd

# Definir las rutas de los archivos CSV
archivos_csv = [
      r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2025 COLLOKY F.csv",
    r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2024 COLLOKY F.csv",
    r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2023 COLLOKY 2.csv",
    r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2022 COLLOKY 2.csv",
    r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2021 COLLOKY 2.csv",
    r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2020 COLLOKY 2.csv",
    r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2019 COLLOKY 2.csv",
    r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2016 COLLOKY 2.csv",
    r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2017 COLLOKY 2.csv",
    r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2018 COLLOKY 2.csv"
]

# Leer cada archivo CSV y almacenar en una lista
dfs = []
for archivo in archivos_csv:
    df_temp = pd.read_csv(archivo, low_memory=False)  # Cargar el CSV con low_memory=False
    
    # Convertir todos los nombres de columnas a minúsculas
    df_temp.columns = df_temp.columns.str.lower()
    
    dfs.append(df_temp)
    
    # Imprimir el conteo de filas de cada archivo CSV
    print(f'Archivo: {archivo} - Cantidad de filas: {df_temp.shape[0]}')

# Unir todos los DataFrames en uno solo
df_unificado = pd.concat(dfs, ignore_index=True)

# Imprimir la cantidad total de filas y columnas del DataFrame unificado
print(f'\nCantidad total de filas en el DataFrame unificado: {df_unificado.shape[0]} - Cantidad total de columnas: {df_unificado.shape[1]}')

# Verificar los nombres de las columnas en minúsculas
print(f'Nombres de las columnas en minúsculas: {list(df_unificado.columns)}')


Archivo: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2025 COLLOKY F.csv - Cantidad de filas: 317329
Archivo: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2024 COLLOKY F.csv - Cantidad de filas: 4133983
Archivo: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2023 COLLOKY 2.csv - Cantidad de filas: 3766863
Archivo: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2022 COLLOKY 2.csv - Cantidad de filas: 3885629
Archivo: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2021 COLLOKY 2.csv - Cantidad de filas: 4124811
Archivo: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2020 COLLOKY 2.csv - Cantidad de filas: 3379862
Archivo: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2019 COLLOKY 2.csv - Cantidad de filas: 4777794
Archivo: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2016 COLLOKY 2.csv - Cantidad de filas: 2393628
Archivo: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\2017 COLLOKY 2.csv - Cantidad de filas: 3

In [2]:
import pandas as pd
import dask.dataframe as dd

# Supongamos que ya tienes el DataFrame df_unificado
# df_unificado = ... (tu DataFrame existente)

# Asegurarse de que la columna 'fecha' esté en formato datetime
df_unificado['fecha'] = pd.to_datetime(df_unificado['fecha'])
print("Fecha convertida a datetime:")
print(df_unificado['fecha'].head())

# Convertir la columna 'rut' a minúsculas
df_unificado['rut'] = df_unificado['rut'].str.lower()
print("RUTs convertidos a minúsculas:")
print(df_unificado['rut'].head())

# Filtrar RUTs presentes en el mes base
mes_base_inicio = '2025-01-27'
mes_base_fin = '2025-02-02'
mes_base_ruts = df_unificado[(df_unificado['fecha'] >= mes_base_inicio) & (df_unificado['fecha'] <= mes_base_fin)]['rut'].unique()
print("RUTs en el mes base:")
print(mes_base_ruts[:10])  # Mostrar los primeros 10 RUTs

# Filtrar las transacciones dentro del período base
df_periodo_base = df_unificado[(df_unificado['fecha'] >= mes_base_inicio) & (df_unificado['fecha'] <= mes_base_fin)]

# Obtener la primera fecha de compra dentro del período base para cada RUT
min_fecha_por_rut = df_periodo_base.groupby('rut')['fecha'].min().reset_index()
min_fecha_por_rut['mes_fecha_min'] = min_fecha_por_rut['fecha']
print("Primera fecha de compra por RUT en el período base:")
print(min_fecha_por_rut.head())

# Filtrar fechas anteriores a la primera fecha de compra para cada RUT de manera eficiente
df_temp = df_unificado.merge(min_fecha_por_rut[['rut', 'mes_fecha_min']], on='rut')
print("Datos después del merge con fechas mínimas:")
print(df_temp.head())

# Filtrar filas donde la fecha es anterior a la primera fecha de compra
df_temp = df_temp[df_temp['fecha'] < df_temp['mes_fecha_min']]
print("Datos filtrados con fechas anteriores a la primera compra:")
print(df_temp.head())

# Obtener la máxima fecha anterior a la primera fecha de compra por RUT
df_temp = df_temp.groupby('rut').agg(seg_fecha=('fecha', 'max')).reset_index()
print("Máxima fecha anterior a la primera compra por RUT:")
print(df_temp.head())

# Clasificar los clientes usando la fecha máxima de cada rut
def clasificar_cliente(row):
    if pd.isna(row['seg_fecha']):
        return None  # No clasificar aquí, lo haremos después
    row['mes_fecha_min'] = pd.to_datetime(row['mes_fecha_min'])  # Asegurarse de que mes_fecha_min esté en formato datetime
    if row['seg_fecha'] >= row['mes_fecha_min'] - pd.DateOffset(months=6):
        return 'Cliente Antiguo Activo'
    elif row['seg_fecha'] >= row['mes_fecha_min'] - pd.DateOffset(months=12):
        return 'Cliente Antiguo Pasivo'
    elif row['seg_fecha'] >= row['mes_fecha_min'] - pd.DateOffset(months=24):
        return 'Cliente Antiguo Recuperado'
    else:
        return 'Cliente Antiguo Perdido'

# Aplicar la clasificación a cada RUT
df_temp = df_temp.merge(min_fecha_por_rut[['rut', 'mes_fecha_min']], on='rut')
print("Datos después del segundo merge:")
print(df_temp.head())

df_temp['clasificacion_cliente'] = df_temp.apply(clasificar_cliente, axis=1)
print("Datos con clasificación de clientes:")
print(df_temp.head())

# Combinar df_temp con df_unificado para agregar clasificaciones
df_resultado = pd.merge(df_unificado, df_temp[['rut', 'clasificacion_cliente']], on='rut', how='left')
print("Resultado después de combinar con clasificaciones:")
print(df_resultado.head())

# Asignar 'Cliente Nuevo' a los RUTs que no tienen clasificación previa
df_resultado['clasificacion_cliente'] = df_resultado['clasificacion_cliente'].fillna('Cliente Nuevo')

# Contar clientes por clasificación y calcular el porcentaje
conteo_clientes = df_resultado.groupby('clasificacion_cliente').agg(total_clientes=('rut', 'nunique')).reset_index()
print("Conteo de clientes por clasificación:")
print(conteo_clientes)

# Calcular el total de RUT únicos en el mes base
total_ruts_octubre = len(mes_base_ruts)
print(f"Total de RUT únicos en el mes de octubre 2024: {total_ruts_octubre}")

# Recalcular clientes nuevos
clientes_antiguos_total = conteo_clientes[conteo_clientes['clasificacion_cliente'] != 'Cliente Nuevo']['total_clientes'].sum()
clientes_nuevos = total_ruts_octubre - clientes_antiguos_total

# Actualizar el DataFrame de conteo
conteo_clientes.loc[conteo_clientes['clasificacion_cliente'] == 'Cliente Nuevo', 'total_clientes'] = clientes_nuevos

# Calcular el porcentaje
conteo_clientes['porcentaje'] = (conteo_clientes['total_clientes'] / total_ruts_octubre) * 100
conteo_clientes = conteo_clientes.sort_values(by='total_clientes', ascending=False)

# Mostrar resultados
print("Conteo de clientes por clasificación:")
print(conteo_clientes)
print(f"Total de RUT únicos en el mes de octubre 2024: {total_ruts_octubre}")

# Verificación adicional
print(df_temp[['rut', 'seg_fecha', 'mes_fecha_min', 'clasificacion_cliente']])


Fecha convertida a datetime:
0   2025-02-02
1   2025-01-04
2   2025-02-02
3   2025-01-07
4   2025-01-08
Name: fecha, dtype: datetime64[ns]
RUTs convertidos a minúsculas:
0    15172914-2
1    15362705-3
2    13645170-7
3    16925597-0
4    17918455-9
Name: rut, dtype: object
RUTs en el mes base:
['15172914-2' '13645170-7' '16611410-1' '17055738-7' '26827795-1'
 '19155794-8' '15329559-k' '10434863-7' '17277459-8' '15128903-7']
Primera fecha de compra por RUT en el período base:
          rut      fecha mes_fecha_min
0  10000040-7 2025-02-01    2025-02-01
1  10004354-8 2025-01-28    2025-01-28
2  10004667-9 2025-01-29    2025-01-29
3  10009375-8 2025-01-30    2025-01-30
4  10013106-4 2025-01-28    2025-01-28
Datos después del merge con fechas mínimas:
       fecha            boleta  montoneto    marca            sku  cantidad  \
0 2025-02-02  1507824429584-01    2114.29  COLLOKY  P1JE41508AV24         1   
1 2025-02-02  1507934430144-01    5037.82  COLLOKY  P1JE19706AV25         1   
2 20

In [9]:
# Filtrar solo los RUT de los clientes nuevos que están en el mes base de noviembre
df_new_nov24 = df_resultado[(df_resultado['rut'].isin(mes_base_ruts))][['rut']].drop_duplicates().reset_index(drop=True)

# Mostrar el conteo de RUT únicos de clientes nuevos
total_clientes_nuevos = df_new_nov24['rut'].nunique()
print(f"Total de RUT únicos de clientes nuevos en noviembre 2024: {total_clientes_nuevos}")

Total de RUT únicos de clientes nuevos en noviembre 2024: 19285


In [10]:
# Define el rango de fechas
mes_base_inicio = '2024-01-29'
mes_base_fin = '2024-02-25'

# Filtrar df_unificado por el rango de fechas y el canal
df_unificado_filtrado = df_unificado[(df_unificado['fecha'] >= mes_base_inicio) & 
                                     (df_unificado['fecha'] <= mes_base_fin)]

# Realizar el merge
df_new_nov24_merged = df_unificado_filtrado.merge(df_new_nov24, left_on='rut', right_on='rut')

# Mostrar el total de filas en el DataFrame resultante del merge
total_filas = df_new_nov24_merged.shape[0]

print("Total de filas en el DataFrame resultante del merge de df_unificado_filtrado con df_new_nov24:")
print(total_filas)


Total de filas en el DataFrame resultante del merge de df_unificado_filtrado con df_new_nov24:
47112


In [11]:
import pandas as pd

# Leer un archivo Excel en un DataFrame
ruta_excel = "C:/Users/constanza.perez/Downloads/Lista_Precio_Opaline.xlsx"
df_precios = pd.read_excel(ruta_excel, sheet_name='Lista_Precio_Opaline')

# Convertir la columna 'fecha' a formato datetime
df_new_nov24_merged['fecha'] = pd.to_datetime(df_new_nov24_merged['fecha'])

# Calcular 'monto neto final'
df_new_nov24_merged['monto neto final'] = df_new_nov24_merged['montoneto'] * 1.19

# Realizar un merge de DataFrames y renombrar columnas
df_new_nov24_merged = df_new_nov24_merged.merge(df_precios[['Código Producto', 'Precio Un.']], how='left', left_on='sku', right_on='Código Producto')

# Calcular 'Precio Lleno'
df_new_nov24_merged['Precio Lleno'] = df_new_nov24_merged['cantidad'] * df_new_nov24_merged['Precio Un.']

# Calcular 'Precio Lleno' total por boleta
df_new_nov24_merged['Precio Lleno Boleta'] = df_new_nov24_merged.groupby('boleta')['Precio Lleno'].transform('sum')

# Calcular 'monto neto final' total por boleta
df_new_nov24_merged['Monto Neto Final Boleta'] = df_new_nov24_merged.groupby('boleta')['monto neto final'].transform('sum')

# Calcular 'descuento'
df_new_nov24_merged['descuento'] = 1 - (df_new_nov24_merged['Monto Neto Final Boleta'] / df_new_nov24_merged['Precio Lleno Boleta'])

# Calcular el descuento promedio por rut
descuento_promedio_por_rut = df_new_nov24_merged.groupby('rut')['descuento'].mean().reset_index()
descuento_promedio_por_rut.columns = ['rut', 'Descuento Promedio']

# Unir el descuento promedio al DataFrame original
df_new_nov24_merged = df_new_nov24_merged.merge(descuento_promedio_por_rut, on='rut', how='left')

# Agregar las columnas 'fecha_min' y 'fecha_max' para cada 'rut'
df_new_nov24_merged['fecha_min'] = df_new_nov24_merged.groupby('rut')['fecha'].transform('min')
df_new_nov24_merged['fecha_max'] = df_new_nov24_merged.groupby('rut')['fecha'].transform('max')

# Asegurarse de que las fechas min y max están en formato datetime
df_new_nov24_merged['fecha_min'] = pd.to_datetime(df_new_nov24_merged['fecha_min'])
df_new_nov24_merged['fecha_max'] = pd.to_datetime(df_new_nov24_merged['fecha_max'])

# Calcular el número de años transcurridos
df_new_nov24_merged['years_diff'] = ((df_new_nov24_merged['fecha_max'] - df_new_nov24_merged['fecha_min']).dt.days / 365).apply(lambda x: int(x) + 1)

# Agrupar por 'rut' y sumar todas las boletas únicas
df_yearly = df_new_nov24_merged.groupby('rut')['boleta'].nunique().reset_index()
df_yearly.columns = ['rut', 'total_boletas']

# Convertir 'years_diff' a meses
df_yearly['years_diff_months'] = df_new_nov24_merged.groupby('rut')['years_diff'].transform('max') * 12

# Calcular la frecuencia mensual de compras únicas
df_yearly['frecuencia_mensual'] = (df_yearly['total_boletas'] / df_yearly['years_diff_months'])*12

# Unir la frecuencia mensual con el DataFrame original
df_final = df_new_nov24_merged.merge(df_yearly[['rut', 'frecuencia_mensual']], on='rut', how='left')

# Calcular el tiempo entre compras
df_final = df_final.sort_values(by=['rut', 'fecha'])
df_final['days_since_last_purchase'] = df_final.groupby('rut')['fecha'].diff().dt.days

# Calcular TPC (Tiempo Promedio entre Compras)
df_tpc = df_final[df_final['days_since_last_purchase'] > 1].groupby('rut')['days_since_last_purchase'].mean().reset_index()
df_tpc.columns = ['rut', 'TPC']

# Unir el TPC con el DataFrame original
df_final = df_final.merge(df_tpc, on='rut', how='left')

# Calcular el ticket promedio (monto neto final total por boleta agrupado por rut)
ticket_promedio_por_rut = df_new_nov24_merged.groupby('rut')['Monto Neto Final Boleta'].mean().reset_index()
ticket_promedio_por_rut.columns = ['rut', 'ticket prom']

# Unir el ticket promedio al DataFrame final
df_final = df_final.merge(ticket_promedio_por_rut, on='rut', how='left')


In [12]:
import pandas as pd

# Establecer 'descuento' como 0 si es menor que 0
df_final.loc[df_final['descuento'] < 0, 'descuento'] = 0

# Imprimir el DataFrame final
(df_final.head())

,fecha,boleta,montoneto,marca,sku,cantidad,area,temporada,correo,genero,...,Monto Neto Final Boleta,descuento,Descuento Promedio,fecha_min,fecha_max,years_diff,frecuencia_mensual,days_since_last_purchase,TPC,ticket prom
0,2024-02-21,25496327,13521.0,OPALINE,4230021021S24,1,CALZADO,VERANO2024,NaN,GIRL,...,16089.99,0.356143,0.356143,2024-02-21,2024-02-21,1,1.0,NaN,NaN,16089.99
1,2024-02-12,25438903,8563.0,OPALINE,STPU481006S22,1,ROPA,VERANO2022,maria1223@gmail.com,GIRL,...,23670.29,0.359743,0.359743,2024-02-12,2024-02-12,1,1.0,NaN,NaN,23670.29
2,2024-02-12,25438903,5874.0,OPALINE,C9PE063002W24,1,ACCESORIOS,INVIERNO2024,maria1223@gmail.com,GIRL,...,23670.29,0.359743,0.359743,2024-02-12,2024-02-12,1,1.0,0.0,NaN,23670.29
3,2024-02-12,25438903,5454.0,OPALINE,BYJE314006S24,1,ROPA,VERANO2024,maria1223@gmail.com,BOY,...,23670.29,0.359743,0.359743,2024-02-12,2024-02-12,1,1.0,0.0,NaN,23670.29
4,2024-02-02,25384271,9992.0,OPALINE,BCTO0550TUS24,1,ACCESORIOS,VERANO2024,mimitocandia@gmail.com,BOY,...,11890.48,0.300148,0.300148,2024-02-02,2024-02-02,1,1.0,NaN,NaN,11890.48


In [ ]:
import pandas as pd

# Función de categorización para la columna 'area'
def categorize_area(column_values):
    unique_values = column_values.unique()
    if len(unique_values) == 1:
        if 'CALZADO' in unique_values:
            return 'SOLO CALZADO'
        if 'ACCESORIOS' in unique_values:
            return 'SOLO ACCESORIO'
        if 'ROPA' in unique_values:
            return 'SOLO ROPA'
    if all(item in unique_values for item in ['CALZADO', 'ACCESORIOS', 'ROPA']):
        return 'MIXTO'
    elif 'CALZADO' in unique_values and 'ACCESORIOS' in unique_values:
        return 'MIXTO'
    elif 'CALZADO' in unique_values and 'ROPA' in unique_values:
        return 'MIXTO'
    elif 'ACCESORIOS' in unique_values and 'ROPA' in unique_values:
        return 'MIXTO'
    else:
        return 'MIXTO'

# Función de categorización para las columnas 'genero' y 'canal'
def categorize_other(column_values):
    unique_values = column_values.unique()
    if 'ACCESORIOS' in unique_values:
        return 'ROPA'
    if len(unique_values) == 1:
        return unique_values[0]
    else:
        return 'MIXTO'

# Aplicar las transformaciones sin perder columnas
df_final['area_categorizada'] = df_final.groupby('rut')['area'].transform(categorize_area)
df_final['genero_categorizado'] = df_final.groupby('rut')['genero'].transform(categorize_other)
df_final['canal_categorizado'] = df_final.groupby('rut')['canal'].transform(categorize_other)

# Eliminar duplicados para obtener 'rut' único
df_result = df_final.drop_duplicates(subset='rut').reset_index(drop=True)

# Imprimir el DataFrame final agrupado y categorizado
print(df_result.head())


In [8]:
import pandas as pd
from IPython.display import display, HTML

# Redondear los valores de frecuencia_mensual
df_result['frecuencia_mensual_redondeada'] = df_result['frecuencia_mensual'].apply(lambda x: round(x))

# Asignar rangos directamente según los valores redondeados de frecuencia_mensual
df_result['frecuencia_mensual_rango'] = df_result['frecuencia_mensual_redondeada'].apply(lambda x: str(x) if x <= 10 else 'Mayor a 10')

# Crear rangos para la variable TICKET PROMEDIO
df_result['ticket_promedio_rango'] = pd.cut(df_result['ticket prom'], bins=[0, 15000, 30000, 45000, 60000, float('inf')],
                                            labels=['0-15000', '15001-30000', '30001-45000', '45001-60000', 'Mayor a 60000'])

# Crear rangos para la variable edad, excluyendo valores nulos
df_result['edad_rango'] = pd.cut(df_result['edad'].dropna(), bins=[0, 18, 30, 40, 50, 60, float('inf')],
                                 labels=['0-18', '19-30', '31-40', '41-50', '51-60', 'Mayor a 60'])

# Ajustar rangos para la variable descuento según tu requerimiento
df_result['descuento_rango'] = pd.cut(df_result['descuento'], bins=[-0.01, 0.20, 0.40, 0.60, float('inf')],
                                      labels=['0%-20%', 'Mayor a 20% - 40%', 'Mayor a 40% - 60%', 'Mayor a 60%'])

# Crear rangos para la variable TPC
df_result['tpc_rango'] = pd.cut(df_result['TPC'], bins=[0, 1, float('inf')],
                                labels=['0-1', 'Mayor a 1'])

# Crear rangos para la variable edad_ano_talla_hoy, excluyendo valores nulos
max_edad_ano_talla_hoy = int(df_result['edad_ano_talla_hoy'].max())  # Convertir a entero
df_result['edad_ano_talla_hoy_rango'] = pd.cut(df_result['edad_ano_talla_hoy'].dropna(), 
                                               bins=range(0, max_edad_ano_talla_hoy + 2),
                                               labels=[str(i) for i in range(0, max_edad_ano_talla_hoy + 1)])

# Función para formatear los porcentajes sin decimales ni puntos
def format_percentage_without_decimals(column_values):
    counts = column_values.value_counts()
    return ', '.join([f"{index}: {int(round((count / len(column_values)) * 100))}%" for index, count in counts.items()])

# Calcular los porcentajes de las variables categóricas, excluyendo valores nulos, NaN o vacíos
porcentajes_area = format_percentage_without_decimals(df_result['area_categorizada'].dropna())
porcentajes_genero = format_percentage_without_decimals(df_result['genero_categorizado'].dropna())
porcentajes_canal = format_percentage_without_decimals(df_result['canal_categorizado'].dropna())
porcentajes_frecuencia_mensual = format_percentage_without_decimals(df_result['frecuencia_mensual_rango'].dropna())
porcentajes_ticket_promedio = format_percentage_without_decimals(df_result['ticket_promedio_rango'].dropna())
porcentajes_edad = format_percentage_without_decimals(df_result['edad_rango'])
porcentajes_descuento = format_percentage_without_decimals(df_result['descuento_rango'].dropna())
porcentajes_tpc = format_percentage_without_decimals(df_result['tpc_rango'].dropna())
porcentajes_edad_ano_talla_hoy = format_percentage_without_decimals(df_result['edad_ano_talla_hoy_rango'])

# Calcular las estadísticas descriptivas de las variables numéricas seleccionadas
estadisticas_numericas = df_result[['cantidad', 'edad', 'edad_ano_talla_hoy', 'ticket prom', 'frecuencia_mensual', 'TPC', 'descuento']].describe().transpose()

# Añadir la media de boletas únicas por rut
media_boletas_unicas = df_result.groupby('rut')['boleta'].nunique().mean()

# Crear un DataFrame con las distribuciones y estadísticas
distribuciones = pd.DataFrame({
    'area_categorizada': [porcentajes_area],
    'genero_categorizado': [porcentajes_genero],
    'canal_categorizado': [porcentajes_canal],
    'frecuencia_mensual_rango': [porcentajes_frecuencia_mensual],
    'ticket_promedio_rango': [porcentajes_ticket_promedio],
    'edad_rango': [porcentajes_edad],
    'descuento_rango': [porcentajes_descuento],
    'tpc_rango': [porcentajes_tpc],
    'edad_ano_talla_hoy_rango': [porcentajes_edad_ano_talla_hoy]
})

# Añadir la media de boletas únicas al DataFrame de estadísticas
estadisticas_numericas.loc['mean_unique_boleta_per_rut'] = media_boletas_unicas

# Unir los DataFrames de distribuciones y estadísticas
resultados_generales = pd.concat([estadisticas_numericas, distribuciones], axis=1).fillna('')

# Convertir el DataFrame a HTML para mejor visualización
display(HTML(resultados_generales.to_html(escape=False)))


,count,mean,std,min,25%,50%,75%,max,area_categorizada,genero_categorizado,canal_categorizado,frecuencia_mensual_rango,ticket_promedio_rango,edad_rango,descuento_rango,tpc_rango,edad_ano_talla_hoy_rango
cantidad,18965.0,1.024941,0.179232,-1.0,1.0,1.0,1.0,6.0,,,,,,,,,
edad,18673.0,34.820222,14.082725,0.0,26.0,32.0,42.0,89.0,,,,,,,,,
edad_ano_talla_hoy,18252.0,2.890861,1.638374,1.0,2.0,3.0,4.0,17.0,,,,,,,,,
ticket prom,18965.0,23016.703382,17683.223806,-11947.911362,11990.44,18579.47,28545.958,310933.0532,,,,,,,,,
frecuencia_mensual,18965.0,1.164566,1.414836,1.0,1.0,1.0,1.0,180.0,,,,,,,,,
TPC,1162.0,8.510772,5.476369,2.0,4.0,7.0,11.0,27.0,,,,,,,,,
descuento,18965.0,0.365734,0.202395,0.0,0.222463,0.400401,0.500343,0.820508,,,,,,,,,
mean_unique_boleta_per_rut,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,,,,,,,,,
0,,,,,,,,,"MIXTO: 31%, SOLO CALZADO: 30%, SOLO ROPA: 23%, SOLO ACCESORIO: 16%","GIRL: 50%, BOY: 36%, MIXTO: 13%, Sin Genero: 2%","OFFLINE: 90%, ONLINE: 9%, MIXTO: 1%","1: 88%, 2: 10%, 3: 2%, 4: 0%, 5: 0%, 6: 0%, Mayor a 10: 0%, 9: 0%, 7: 0%, 10: 0%, 8: 0%","15001-30000: 40%, 0-15000: 38%, 30001-45000: 13%, 45001-60000: 5%, Mayor a 60000: 4%","19-30: 35%, 31-40: 31%, 41-50: 11%, 51-60: 8%, Mayor a 60: 7%, 0-18: 5%","Mayor a 40% - 60%: 44%, Mayor a 20% - 40%: 21%, 0%-20%: 21%, Mayor a 60%: 14%","Mayor a 1: 100%, 0-1: 0%","0: 24%, 1: 23%, 2: 15%, 3: 15%, 4: 13%, 5: 6%, 6: 0%, 11: 0%, 8: 0%, 7: 0%, 10: 0%, 9: 0%, 16: 0%, 12: 0%, 13: 0%, 14: 0%, 15: 0%, 17: 0%"
